# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

from math import radians, cos, sin, asin, sqrt

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import pyarrow.parquet as pq
import re
import numpy as np
import sys
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
# any general notebook setup, like log formatting
!pip install pyarrow
!pip install geopandas

In [3]:
#urls
TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

#local csvs
UBER_DATA = "uber_rides_sample.csv"
WEATHER_CSV = ["2015_weather.csv","2014_weather.csv","2013_weather.csv","2012_weather.csv","2011_weather.csv","2010_weather.csv","2009_weather.csv"]

#coords
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** to find the trip distance by transfering coordinates (degrees) into the distance in kilometers._

In [4]:
uber_rides_sample = pd.read_csv("uber_rides_sample.csv")

In [5]:
def calculate_distance(from_coord, to_coord):
    
    #transfer from degrees to radians.
    #get latitude and longitude of the from_coord
    pick_lat = from_coord["pickup_latitude"].map(radians)
    pick_long = from_coord["pickup_longitude"].map(radians)
    #get latitude and longitude of the to_coord
    drop_lat = to_coord["dropoff_latitude"].map(radians)
    drop_long = to_coord["dropoff_longitude"].map(radians)
    
    distance_long = pick_long - drop_long
    distance_lat = pick_lat - drop_lat
    
    a = ((distance_lat / 2).map(sin))**2 + (pick_lat).map(cos) * (drop_lat).map(cos) * ((distance_long / 2).map(sin))**2
    c = 2 * (a.map(sqrt)).map(asin)
    r = 6378 # Radius of earth in kilometers
    
    return c*r


In [6]:
def add_distance_column(dataframe):
    #add the 'distance' column into the dataframe
    #distance is the result of calculate_distance(from_coord, to_coord)
    from_coord = dataframe[["pickup_latitude","pickup_longitude"]]
    to_coord = dataframe[["dropoff_latitude","dropoff_longitude"]]
    
    dataframe["distance"] = calculate_distance(from_coord, to_coord)
    return dataframe

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [7]:
def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html
    
def find_taxi_parquet_links():
    html = get_taxi_html()
    soup = bs4.BeautifulSoup(html, "html.parser")
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    result = [a["href"] for a in yellow_a_tags]
    pattern = re.compile(
    r"yellow_tripdata_20((15-0([1-6]))|(09-(\d{2}))|(1[0-4]-(\d{2}))).parquet"
    )
    links = []
    for link in result:
        match = pattern.search(link)
        if match:
            links.append(match.string) 
    return links

In [8]:
taxi_zones = gpd.read_file(filename='taxi_zones.zip', engine='fiona')
taxi_zones = taxi_zones.to_crs(4326)
taxi_zones['longitude'] = taxi_zones.centroid.x
taxi_zones['latitude'] = taxi_zones.centroid.y
# Remove duplicated location IDs
taxi_zones.drop_duplicates('LocationID',inplace=True)
taxi_zones.set_index('LocationID',inplace=True)

In [9]:
def get_and_clean_month_taxi_data(url):

    df = pd.read_parquet(url)


    if 'PULocationID' in df.columns:
        df['pickup_latitude'] = df['PULocationID'].map(taxi_zones['latitude'])
        df['pickup_longitude'] = df['PULocationID'].map(taxi_zones['longitude'])
        df['dropoff_latitude'] = df['DOLocationID'].map(taxi_zones['latitude'])
        df['dropoff_longitude'] = df['DOLocationID'].map(taxi_zones['longitude'])
        
    # Set all column names to lower case    
    df = df.rename(columns=str.lower)
    
    # Normalize column names
    df.rename(columns={'tpep_pickup_datetime':'pickup_datetime','trip_pickup_datetime':'pickup_datetime','tpep_dropoff_datetime':'dropoff_datetime','trip_dropoff_datetime':'dropoff_datetime','start_lon':'pickup_longitude', 'start_lat':'pickup_latitude', 'end_lon':'dropoff_longitude', 'end_lat':'dropoff_latitude'}, inplace=True)
    
    # Remove invalid data points
    df.dropna(subset=['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude'],inplace=True)
    #df = df[(df['payment_type']== "CASH") | (df['payment_type']== "Credit")]
    df = df[df['passenger_count']>=1]
    
    # Remove unnecessary columns
    df = df[['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]

    # Remove trips that start and/or end outside the designated coordinate box (NEW_YORK_BOX_COORDS)
    df = df[(df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & (df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0])]
    df = df[(df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & (df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]

    # Sample
    df = df.sample(n=2564)

    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df = df.astype({'pickup_longitude':'float32','pickup_latitude':'float32','dropoff_longitude':'float32','dropoff_latitude':'float32'})

    return df

In [10]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_parquet_urls = find_taxi_parquet_links()
    for parquet_url in tqdm(all_parquet_urls):
        dataframe = get_and_clean_month_taxi_data(parquet_url)
        add_distance_column(dataframe)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    
    # Remove unnecessary columns
    df = df[['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
    
    # Remove trips that start and/or end outside the designated coordinate box (NEW_YORK_BOX_COORDS)
    df = df[(df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & (df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0])]
    df = df[(df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & (df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]
    
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df = df.astype({'pickup_longitude':'float32','pickup_latitude':'float32','dropoff_longitude':'float32','dropoff_latitude':'float32'})
    return df

In [12]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [13]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    
    #find the required columns
    df = df[["DATE","HourlyPrecipitation", "HourlyWindSpeed"]]
    
    #normalize the 'DATE' columns into datetime form (dtype:int8)
    df["DATE"] = pd.to_datetime(df["DATE"])
    
    #normalize the 'Precipitation' columns into numeric form (dtype: float32) and fillout the blank with value 0
    df["HourlyPrecipitation"].fillna(value = 0, inplace = True)
    df[df["HourlyPrecipitation"]=="T"] = float(0.00001)
    df[df["HourlyPrecipitation"]=="M"] = np.nan
    df['HourlyPrecipitation'] = pd.to_numeric(df['HourlyPrecipitation'], errors = "coerce")
    
    #fill out the missing blank for 'WindSpeed' with 0
    df["HourlyWindSpeed"].fillna(value = 0, inplace = True)
    
    df = df.astype({"HourlyWindSpeed":"int8","HourlyPrecipitation":"float32"})
    
    return df

In [14]:
def clean_month_weather_data_daily(csv_file):
    df = clean_month_weather_data_hourly(csv_file)
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce').dt.date
    df_windspeed = df.groupby('DATE')['HourlyWindSpeed'].mean()
    df_precipitation = df.groupby('DATE')['HourlyPrecipitation'].sum()
    
    df = pd.merge(df_windspeed,df_precipitation,on='DATE')
    df = df.rename(columns={'HourlyWindSpeed': 'DailyWindSpeed', 'HourlyPrecipitation': 'DailyPrecipitation'})
    df = df.astype({"DailyWindSpeed":"int8","DailyPrecipitation":"float32"})
    return df

In [15]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["2015_weather.csv","2014_weather.csv","2013_weather.csv","2012_weather.csv","2011_weather.csv","2010_weather.csv","2009_weather.csv"]
    
    for csv_file in tqdm(weather_csv_files):
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [16]:
taxi_data = get_and_clean_taxi_data()

100%|██████████| 78/78 [50:33<00:00, 38.89s/it]


In [17]:
uber_data = get_uber_data()

In [18]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

100%|██████████| 7/7 [00:01<00:00,  4.02it/s]


## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [20]:
engine = db.create_engine(DATABASE_URL)

In [21]:
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    HourlyWindSpeed INTETER,
    HourlyPrecipitation FLOAT
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    DailyWindSpeed INTETER,
    DailyPrecipitation FLOAT
)
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATE,
    PULocationID INTETER
    DOLocationID INTETER
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATE,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

In [22]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [23]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [24]:
def write_dataframes_to_table(table_to_df_dict):
    for table, df in table_to_df_dict.items():
        df.to_sql(table, con=engine, if_exists="append", index=False)

In [25]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

NameError: name 'hourly_data' is not defined

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)